# Workshop 2: Introduction to `Snakemake`, new features update & benchmarking

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pypsa atlite pandas geopandas xarray matplotlib hvplot geoviews plotly highspy holoviews folium mapclassify snakemake
```
:::

In [ ]:
# uncomment for running this notebook on Colab
# !pip install pypsa atlite pandas geopandas xarray matplotlib hvplot geoviews plotly highspy holoviews folium mapclassify snakemake

In [ ]:
# import packages
from IPython.display import Code, SVG, Image, display
from urllib.request import urlretrieve
from matplotlib.ticker import MultipleLocator
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import os
import zipfile
import numpy as np
import pypsa
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pypsa.plot.maps.static import (
    add_legend_circles,
    add_legend_patches,
    add_legend_lines,
)

pypsa.options.params.statistics.round = 3
pypsa.options.params.statistics.drop_zero = True
pypsa.options.params.statistics.nice_names = False

In [ ]:
urls = {
    "data/data_raw.csv": "https://storage.googleapis.com/open-tyndp-data-store/workshop-02/data_raw.csv",
    "data/open-tyndp.zip": "https://storage.googleapis.com/open-tyndp-data-store/workshop-02/open-tyndp.zip",
    "data/network_NT_presolve_highres_2030.nc": "https://storage.googleapis.com/open-tyndp-data-store/workshop-02/network_NT_presolve_highres_2030.nc",
}

os.makedirs("data", exist_ok=True)
for name, url in urls.items():
    if os.path.exists(name):
        print(f"File {name} already exists. Skipping download.")
    else:
        print(f"Retrieving {name} from GCP storage.")
        urlretrieve(url, name)
        print(f"File available in {name}.")

to_dir = "data/open-tyndp"
if not os.path.exists(to_dir):
    with zipfile.ZipFile("data/open-tyndp.zip", "r") as zip_ref:
        zip_ref.extractall(to_dir)
print(f"Open-TYNDP available in '{to_dir}'.")

print("Done")

# The `Snakemake` tool

![](snakemake_logo.png)

The `Snakemake` workflow management system is a tool to create reproducible and scalable data analyses.
Workflows are described via a human readable, Python based language. They can be seamlessly scaled to server, cluster, grid, and cloud environments, without the need to modify the workflow definition.

Snakemake follows the [GNU Make](https://www.gnu.org/software/make) paradigm: workflows are defined in terms of so-called `rules` that define how to create a set of output files from a set of input files. Dependencies between the rules are determined automatically, creating a DAG (directed acyclic graph) of jobs that can be automatically parallelized.

:::{note}
Documentation for this package is available at https://snakemake.readthedocs.io/. You can also check out a [slide deck Snakemake Tutorial](https://slides.com/johanneskoester/snakemake-tutorial) by Johannes Köster (2024).

Mölder, F., Jablonski, K.P., Letcher, B., Hall, M.B., Tomkins-Tinch, C.H., Sochat, V., Forster, J., Lee, S., Twardziok, S.O., Kanitz, A., Wilm, A., Holtgrewe, M., Rahmann, S., Nahnsen, S., Köster, J., 2021. Sustainable data analysis with Snakemake. F1000Res 10, 33.
:::


## A minimal Snakemake example

To check out how this looks in practice, we've prepared a minimal Snakemake example workflow that processes some data. The minimal workflow consists of the following rules:
- `retrieve_data`
- `build_data`
- `prepare_network`
- `solve_network`
- `plot_benchmark`
- `all`

<center>

![](minimal_workflow.png)

</center>

We will first need to load the raw data file used in this minimal example into our working directory:

### The `Snakefile` and `rules`

The rules need to be defined in a so-called `Snakefile` that sits in your current working directory. For our minimal example the `Snakefile` looks like this:

In [ ]:
Code(filename="Snakefile", language="Python")

### Calling a workflow

You can then execute the workflow by specifying the target file `data/benchmark.pdf` or any intermediate file:
```
snakemake -call data/benchmark.pdf
```

Alternatively you can also execute the workflow by calling a rule that produces an intermediate file:
```
snakemake -call build_data
```
**NOTE:** You cannot call a rule that includes a wildcard without specifying what the wildcard should be filled with. Otherwise Snakemake will not know what to propagate back.

Or you can call the common rule `all` which can be used to execute the entire workflow. It takes the final workflow output as its input and thus requires all previous dependent rules to be run as well:
```
snakemake -call all
```

A very important instrument is the `-n` flag which executes a `dry-run`. It is recommended to always first execute a `dry-run` before the actual execution of the workflow. This simply prints out the directed acyclic graph (DAG) of the workflow to investigate without actually executing it.

Let's try this out and investigate the output:

In [ ]:
! snakemake -call all -n

### Visualizing the `DAG` of a workflow

You can also visualize the `DAG` of jobs using the `--dag` flag and the Graphviz `dot` command. This will not run the workflow but only create the visualization:
```
snakemake -call all --dag | dot -Tsvg > dag.svg
```

In [ ]:
! snakemake -call all --dag | sed -n "/digraph/,\$p" | dot -Tpng > dag_minimal.png

In [ ]:
display(Image("dag_minimal.png"))

Alternatively, you can also visualize a filegraph like the figure above which also includes some information about the inputs and outputs to each of the rules.

You can reproduce the figure from above with the following command:
```
snakemake -call all --filegraph | dot -Tsvg > filegraph.svg
```

In [ ]:
! snakemake -call all --filegraph | sed -n "/digraph/,\$p" | dot -Tsvg > filegraph_minimal.svg

In [ ]:
display(SVG("filegraph_minimal.svg"))

## Task 1: Executing a workflow with Snakemake

a) For our minimal example, execute a `dry-run` to produce the intermediate file `data/base_2030.nc`.

b) Execute the workflow and investigate what happens if you try to execute the workflow again.

c) Delete the final output files `data/benchmark.pdf` and investigate what happens if you try to execute the workflow again.

d) Import the raw input data file `data/data_raw.csv` using pandas and save it again overwriting the original file. Investigate what happens if you try to execute the workflow again. <br>
Hint: Alternatively you can also just `touch` the file by executing `from pathlib import Path` and `Path("data/data_raw.csv").touch()`

e) Finally, open the `Snakefile` and add a second rule that processes the file `data_raw_2.csv` using the same script as the `build_data` rule. Add the output of this new rule as a second input to the `prepare_network` rule.

## Using Snakemake to launch the open-TYNDP workflow

We have already retrieved a prebuilt version of the `open-tyndp` GitHub repository into our working directory.

The `open-tyndp` repository contains the following structure. Directories of particular interest are marked in bold:

- **benchmarks**: will store snakemake benchmarks (does not exist initially)
- **config**: configurations used in the study
- cutouts: will store raw weather data cutouts from atlite (does not exist initially)
- **data**: includes input data that is not produced by any snakemake rule. Various different input files are retrieved from external storage and stored in this directory
- doc: includes all files necessary to build the readthedocs documentation of PyPSA-Eur
- **envs**: includes all the mamba environment specifications to run the workflow
- logs: will store log files (does not exist initially)
- **notebooks**: includes all the notebooks used for ad-hoc analysis
- report: contains all files necessary to build the report; plots and result files are generated automatically
- **rules**: includes all the snakemake rules loaded in the Snakefile
- **resources**: will store intermediate results of the workflow which can be picked up again by subsequent rules (does not exist initially)
- **results**: will store the solved PyPSA network data, summary files and output plots (does not exist initially)
- **scripts**: includes all the Python scripts executed by the snakemake rules to build the model

## Task 2: Explore the folder

a) Can you find the TYNDP specific data input files?

b) Where can you verify the scenario and planning horizons used to produce the current results?

(hint: search for `config.tyndp.yaml`)

c) Can you find the hydrogen grid map in the outputs files for the NT scenario in 2040?

(hint: search for `base_s_all__-h2_network_2040.pdf`)

---


We now need to change our working directory to this new directory:

In [ ]:
os.chdir("data/open-tyndp")

Let's check that we are indeed in the new directory now:

In [ ]:
os.getcwd()

We can now use Snakemake to call some of the rules to produce outputs with the `open-tyndp` PyPSA model. 

We will use the prepared TYNDP configuration file (`config/config.tyndp.yaml`) and schedule a dry-run with `-n` as we only want to investigate the DAG of the workflow:

In [ ]:
! snakemake -call all --configfile config/config.tyndp.yaml -n

In [ ]:
# Touch files to show snakemake logic

The corresponding rule graph to this workflow will look like this:

In [ ]:
! snakemake -call all -F --rulegraph | sed -n "/digraph/,\$p" | dot -Tpng > rulegraph_open_tyndp.png

In [ ]:
display(Image("rulegraph_open_tyndp.png"))

As you can see, this workflow is much more complicated than our minimal example from the beginning.

However, the general idea remains the same. We retrieve data which we consequently process, then we prepare the model network and we solve it before we postprocess the results (summary, plotting, benchmarks).

:::{note}
If you are executing this notebook on your local machine, you can also use the `conda` package manager to install the `open-tyndp` environment and run the workflow instead of dry-runs:
```
conda env create --file envs/<YourSystemOS>-pinned.yaml
```
:::

## Task 3: Adjusting the Open-TYNDP workflow with the configuration file

a) Make some changes in the configuration file and call another **dry-run** of the `open-tyndp` model again to see the changes to the workflow.

Disabling all sectors, for instance, reduces the number of rules to be executed.

In [ ]:
# TODO: Improve

In [ ]:
! snakemake -call --configfile config/config.tyndp.yaml -n

# Update on new features

For the purposes of this workshop, we will primarily focus on the National Trends (NT) scenario when applicable. Four major features were introduced since our last workshop:
1. Addition of the electricity demand and PECD capacity factors time series,
2. Addition of onshore wind and solar TYNDP technologies (incl. PEMMDB existing capacities and trajectories),
3. Addition of offshore hubs (incl. the offshore topology, all associated technologies, potential constraints and trajectories),
4. Addition of hydrogen import corridors.

# Time Series

In [ ]:
# look for renewables components
# Explain / remind difference between time varying and fixed attributes, how to access them
# Plot both time of parameters

The Open-TYNDP data we retrieved contains a network with low time resolution. This is illustrative; however, since we are focusing on time series, we will use another higher resolution network. So, we will import a high resolution pre-solved network for the National Trends (NT) scenario for 2030.

In [ ]:
# hourly networks
n_NT_2030h = pypsa.Network("../network_NT_presolve_highres_2030.nc")

## Electricity demand

We can then explore the electricity demand that is attached to the network. Can you remember how to access `Loads` time series in PyPSA?

Correct! You can use the `loads_t` key and its `p_set` attribute:

In [ ]:
loads_2030 = n_NT_2030h.loads_t.p_set
loads_2030.head()

Let's plot the electricity demand time series:

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))
loads_2030.div(1e3).plot(
    xlabel="Time",
    ylabel="Load [GW]",
    title="Electricity Load Time Series - NT - 2030",
    grid=True,
    ax=ax,
)

ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.15), ncols=10)
ax.grid(True, linestyle="--");

This is very confusing to look at. Let's filter that down a bit...

In [ ]:
# group country profiles together and select a week
country_mapping = n_NT_2030h.buses.query("carrier=='AC'").country
loads_2030_by_country = (
    n_NT_2030h.loads_t.p_set.T.rename(country_mapping, axis=0)
    .groupby("Load")
    .sum()
    .T.loc["2009-03-01":"2009-03-07", ["FR", "DE", "GB"]]
)

# Create the plot
fig, ax = plt.subplots(figsize=(14, 7))
loads_2030_by_country.div(1e3).plot(
    xlabel="Time",
    ylabel="Load [GW]",
    title="Electricity Load Time Series - NT - 2030",
    ax=ax,
)

ax.grid(True, linestyle="--")
ax.legend();

## Task 4: Use PyPSA statistics to explore the load

Can you remember how to use the **PyPSA Statistics** module that we introduced in the last workshop to interactively visualize these electricity demand inputs from the network:

First, we need to load a lower-resolution network that was solved, so we can use the statistics module to analyze it.

In [ ]:
n_NT_2030 = pypsa.Network("results/tyndp/NT/networks/base_s_all___2030.nc")
n_NT_2030.carriers.loc["none", "color"] = "#000000"
n_NT_2030.carriers.loc["", "color"] = "#000000"

n_DE_2040 = pypsa.Network("results/tyndp/DE/networks/base_s_all___2040.nc")
n_DE_2040.carriers.loc["none", "color"] = "#000000"
n_DE_2040.carriers.loc["", "color"] = "#000000"

In [ ]:
# let's define a helper variable
s = n_NT_2030.statistics

Let's access the data using `s.withdrawal()`.

In [ ]:
s.withdrawal(
    bus_carrier="low voltage", comps="Load", aggregate_time=False, groupby=False
).T.head()

We can also plot all the countries at the same time...

In [ ]:
fig, ax, facet_grid = s.withdrawal.plot.line(
    bus_carrier="low voltage",
    y="value",
    x="snapshot",
    color="country",
)
fig.suptitle("Electricity demand Time Series - NT - 2030", y=1.05)
ax.set_ylabel("Load [MW]")
ax.set_xlabel("Time");

This is once again difficult to read. Let's keep only two countries.

In [ ]:
fig, ax, facet_col = s.withdrawal.plot.area(
    bus_carrier="low voltage",
    y="value",
    x="snapshot",
    color="carrier",
    stacked=True,
    facet_row="country",
    query="carrier == 'electricity' and country in ['DE', 'FR']",
)
fig.suptitle("Electricity demand Time Series - NT - 2030", y=1.05)
ax[0, 0].set_ylabel("Load [MW]")
ax[1, 0].set_ylabel("Load [MW]")
ax[1, 0].set_xlabel("Time");

## PECD capacity factors

The Pan-European Climate Database (PECD) provides capacity factor profiles for all the different renewable technologies used in the TYNDP. We processed these input data files into a Python and PyPSA friendly input format.

Let's start by looking at the processed capacity factor time series for solar PV Utility for 2030. These processed data are stored in the `resources` directory, as they are an output of `build_renewable_profiles_pecd`. We will filter the data to a set of countries and a week.

In [ ]:
cf_pv_rftp = pd.read_csv(
    "resources/tyndp/NT/pecd_data_LFSolarPVRooftop_2030.csv",
    index_col=0,
    parse_dates=True,
).loc["2009-07-01":"2009-07-04", ["SE04", "DE00", "FR00", "ES00"]]
cf_pv_rftp.head()

Using a heatmap, we can better grasp the content of the data.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(cf_pv_rftp.T, cmap="viridis", cbar_kws={"label": "Capacity Factor"}, ax=ax)

tick_positions = range(0, len(cf_pv_rftp), 24)
ax.set_xticks(tick_positions)
ax.set_xticklabels(
    cf_pv_rftp.index[tick_positions].strftime("%Y-%m-%d"), rotation=45, ha="right"
)

ax.set_xlabel("Time")
ax.set_ylabel("Node")
plt.tight_layout()
plt.show()

We can also present the data as time series.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))

cf_pv_rftp.plot(
    title="Capacity Factor Time Series National Trends 2030 - March 1-4",
    xlabel="Date",
    ylabel="Capacity Factor",
    ax=ax,
);

## Task 5: Compute average capacity factor

a) Locate the resource file with onwind capacity factors used for NT in 2030.

b) Compute the average onwind capacity factor for all the countries in the PECD.

c) Verify one of the values directly in the network.

(hint: capacity factors are defined as time varying parameter of generators and are called `p_max_pu`)

In [ ]:
cf_onwind = pd.read_csv(
    "resources/tyndp/NT/pecd_data_Wind_Onshore_2030.csv",
    index_col=0,
    parse_dates=True,
)
cf_onwind.head();

In [ ]:
cf_onwind.columns = cf_onwind.columns.str[:2]
cf_onwind.columns.name = "country"
cf_onwind.head();

In [ ]:
cf_onwind_sorted = cf_onwind.T.groupby(by="country").mean().mean(axis=1).sort_values()
cf_onwind_sorted.tail();

In [ ]:
fig, ax = plt.subplots(figsize=(14, 9))
cf_onwind_sorted.plot.barh(
    title="Average capacity factors - 2030",
    xlabel="Capacity factor [p.u]",
    ylabel="Country",
    ax=ax,
);

In [ ]:
c_buses = n_NT_2030h.buses.query("country=='IE'").index
c_gen = n_NT_2030h.generators.query("carrier=='onwind' and bus in @c_buses").index
c_cf = n_NT_2030h.generators_t.p_max_pu[c_gen]
c_cf.mean()

# Onshore wind and solar

The TYNDP provides expansion trajectories for given investment candidates and expandable technologies. For the implemented onshore wind and solar technologies, these have been included in beta release v0.3 of the Open-TYNDP model.

It is possible to retrieve those values from the network. However, for simplicity, we will import the values directly from the processed input files for the `DE` scenario to investigate the entire trajectory paths at once.

In [ ]:
trajectories = pd.read_csv("resources/tyndp/DE/tyndp_trajectories.csv")
trajectories.head()

Similar to the capacity factor time series, we want to focus on the Solar PV Rooftop technology and its trajectory path. Let's take Germany (DE00) to investigate.

In [ ]:
trajectories_pv_utility_de = (
    trajectories.query("carrier == 'solar-pv-rooftop' and bus == 'DE00'")
    .sort_values(by="pyear")
    .set_index("pyear")[["p_nom_min", "p_nom_max"]]
    .div(1e3)  # GW
)
trajectories_pv_utility_de

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))
trajectories_pv_utility_de.plot(
    title="Solar PV Utility Capacity Trajectories - DE scenario - DE00",
    xlabel="Planning Year",
    ylabel="Capacity [GW]",
    color=["#E63946", "#1D3557"],
    style="*--",
    ax=ax,
)

ax.fill_between(
    trajectories_pv_utility_de.index,
    trajectories_pv_utility_de.iloc[:, 0],
    trajectories_pv_utility_de.iloc[:, 1],
    alpha=0.25,
    color="#457B9D",
    label="Trajectory Range",
)

ax.xaxis.set_major_locator(MultipleLocator(5))

Now, let's access the network for the DE scenario to compare one of these trajectory values for 2040.

In [ ]:
trajectories_pv_utility_de_from_network = n_DE_2040.generators.query(
    "carrier == 'solar-pv-utility' and bus == 'DE00'"
)[["p_nom", "p_nom_min", "p_nom_max"]].div(
    1e3
)  # in GW
trajectories_pv_utility_de_from_network

As we can see, the `p_nom_max` and `p_nom_min` values for 2040 do not match directly with the reported trajectory values analyzed above. This is because each new Generator will have set trajectories that correspond to the new cumulatively installed capacities taking into account optimization results from previous years. Therefore, if we add up the existing capacity (`p_nom`) from 2030 to the `p_nom_max` and `p_nom_min` values from 2040, we will obtain the reported trajectory values shown above:

In [ ]:
(
    trajectories_pv_utility_de_from_network.loc[
        "DE00 0 solar-pv-utility-2040", ["p_nom_min", "p_nom_max"]
    ]
    + trajectories_pv_utility_de_from_network.loc[
        "DE00 0 solar-pv-utility-2030", "p_nom"
    ]
)

## Task 6: Verify onshore wind trajectories

Verify onshore wind trajectories in the network itself. This can be quick if you can copy and reuse the existing code used above.

# Offshore Hubs

To implement the offshore methodology, new carriers (*i.e.*, technologies) are introduced. All the offshore technologies start with `offwind`.

In [ ]:
offwind_carriers = n_NT_2030.carriers.query("Carrier.str.contains('offwind')")
offwind_carriers_i = offwind_carriers.index
offwind_carriers

As you can see in the table above, all the offshore technologies are implemented. We model technologies that are a combination of the following:
- both AC `ac` and DC `dc` zones, as well as H2 generating windfarms `h2`;
- both fixed-bottom `fb` and floating `fl` foundations;
- both radial `r` and hub `oh` connections.

We also introduce new offshore buses, both for electricity and hydrogen. Electricity buses use `AC_OH` as the carrier, while hydrogen buses use `H2_OH`.

In [ ]:
n_NT_2030.buses.query("carrier.str.contains('OH')").head()

Let's narrow down this list to a single country.

In [ ]:
buses = n_NT_2030.buses.query("carrier.str.contains('OH') and country=='BE'")
buses_i = buses.index
buses

Using `n.plot.explore()`, we can easily get an overview of the network topology. Let's clean the network before exploring it to only focus on the electrical offshore topology.

In [ ]:
# let's filter AC_OH buses and explore
n_explore_ac = n_NT_2030.copy()
n_explore_ac.remove("Bus", n_explore_ac.buses.query("carrier not in ['AC_OH']").index)
n_explore_ac.plot.explore()

Now, we can search the network for generators that are defined with those carriers.

In [ ]:
n_NT_2030.generators.query("carrier in @offwind_carriers_i").head()

Let's focus on a specific country:

In [ ]:
n_NT_2030.generators.query("carrier in @offwind_carriers_i and bus in @buses_i")

## Task 7: Extract existing offshore capacities

Extract existing offshore capacities for the country of your choice.

In [ ]:
n_DE_2040.statistics.optimal_capacity(
    bus_carrier=["AC", "AC_OH", "H2_OH"],
    comps="Generator",
    groupby=["bus"],
).to_frame().query("bus.str.contains('BE')")

In [ ]:
fig, ax, facet_grid = n_DE_2040.statistics.optimal_capacity.plot.bar(
    bus_carrier=["AC", "AC_OH", "H2_OH"],
    query="carrier.str.startswith('offwind') and country in ['NL', 'GB']",
    facet_col="country",
)
fig.suptitle("Offshore wind capacities - DE - 2040", y=1.05);

We can also explore the data with maps.

In [ ]:
# Let's clean a network copy to only keep offshore data
n_map = n_DE_2040.copy()
n_map.remove("Bus", n_map.buses.query("carrier not in ['AC', 'AC_OH', 'H2_OH']").index)
n_map.remove(
    "Generator", n_map.generators.query("not carrier.str.startswith('offwind')").index
)
n_map.remove("Link", n_map.links.index)
n_map.remove("StorageUnit", n_map.storage_units.index)

In [ ]:
# Define map projection
def load_projection(plotting_params):
    proj_kwargs = plotting_params.get("projection", dict(name="EqualEarth"))
    proj_func = getattr(ccrs, proj_kwargs.pop("name"))
    return proj_func(**proj_kwargs)


proj = load_projection(dict(name="EqualEarth"))

In [ ]:
# Create the map
subplot_kw = {"projection": proj}
fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=subplot_kw)
n_map.statistics.optimal_capacity.plot.map(
    bus_carrier=["AC", "AC_OH", "H2_OH"],
    ax=ax,
    bus_area_fraction=0.002,
    title="Offshore wind capacities - DE - 2040",
);

The NT scenario is a dispatch scenario. This is translated in PyPSA with the argument `p_nom_extendable = False`. However, for the two other scenario, we need to model capacity expansion. 

Currently, the model is configured to do myopic optimisaiton. This means that only the capacities of the current planning horizon are expandable. Generators of the previous planning horizon are not. Let's check this the network.

In [ ]:
# Let's explore offshore wind generators in Danemark
c_buses = n_DE_2040.buses.query("country == 'DK'").index
(
    n_DE_2040.generators.query("carrier in @offwind_carriers_i and bus in @c_buses")[
        ["p_nom", "p_nom_min", "p_nom_max", "p_nom_opt", "p_nom_extendable"]
    ]
)

As you can in the table, multiple values exists:
- `p_nom`, the nominal power
- `p_nom_min`, if p_nom is extendable, the minimal value
- `p_nom_max`, if p_nom is extendable, the maximal value
- `p_nom_opt`, the optimised nominal power

The `p_nom_min` reflects the exesting capacities defined in the TYNDP, while the `p_nom_max` represent the layer potential. We also implemented constraints to ensure to respect the zone potentials and the trajectories defined in the data:
- A constraint limits the expansion of DC and H2 sitting on the same location, as the sum of the two capacities cannot exceed the layer potential.
- A constraint sets the maximum potential per zone, taking into account the zone trajectories.

# H2 imports

In [ ]:
# read network
# plot H2 map with import corridors
# table to showcase the corridors
# optional: Task to investigate values

There have also been some important additions to the H2 infrastructure since our last workshop. The different H2 import corridors are now included in the model with a simple pipeline transport representation, similar to the H2 reference grid.

We can investigate our National Trends network to create a similar plot to what we showed last time. Let's define some (quite extensive) plotting functions from the open-tyndp workflow:

In [ ]:
def group_import_corridors(df):
    """
    Group pipes which connect same buses and return overall capacity.
    """
    df = df.copy()

    # there are pipes for each investment period rename to AC buses name for plotting
    df["index_orig"] = df.index
    df.rename(index=lambda x: x.split(" - ")[0], inplace=True)
    return df.groupby(level=0).agg(
        {"p_nom": "sum", "p_nom_opt": "sum", "index_orig": "first"}
    )


def plot_h2_map_base(
    network,
    map_opts,
    proj,
    figsize=(12, 12),
    expanded=False,
    regions_for_storage=None,
    color_h2_pipe="#499a9c",
    color_h2_imports="#FFA500",
    color_h2_node="#ff29d9",
):
    """
    Plots the base hydrogen network pipelines capacities, hydrogen buses and import potentials.
    If expanded is enabled, the optimal capacities are plotted instead.
    If regions are given, hydrogen storage capacities are plotted for those regions with aggregated H2 tank storage
    and underground H2 cavern capacities.

    Parameters
    ----------
    network : pypsa.Network
        PyPSA network for plotting the hydrogen grid. Can be either presolving or post solving.
    map_opts : dict
        Map options for plotting.
    expanded : bool, optional
        Whether to plot expanded capacities. Defaults to plotting only base network (p_nom).
    regions_for_storage : gpd.GeoDataframe, optional
        Geodataframe of regions to use for plotting hydrogen storage capacities. Index needs to match storage locations.
        If none is given, no hydrogen storage capacities are plotted.

    Returns
    -------
    None
        Saves the map plot as figure.
    """
    n = network.copy()

    linewidth_factor = 4e3

    n.links.drop(
        n.links.index[
            ~(
                n.links.carrier.str.contains("H2 pipeline")
                | n.links.carrier.str.contains("H2 import")
            )
        ],
        inplace=True,
    )
    n.links.drop(
        n.links.index[n.links.carrier.str.contains("OH")],
        inplace=True,
    )

    p_nom = "p_nom_opt" if expanded else "p_nom"
    # capacity of pipes and imports
    h2_pipes = n.links[n.links.carrier == "H2 pipeline"][p_nom]
    h2_imports = n.links[n.links.carrier.str.contains("H2 import")]

    # group high and low import corridors together
    h2_imports = group_import_corridors(h2_imports)[p_nom]
    n.links.rename(index=lambda x: x.split(" - ")[0], inplace=True)
    # group links by summing up p_nom values and taking the first value of the rest of the columns
    other_cols = dict.fromkeys(n.links.columns.drop(["p_nom_opt", "p_nom"]), "first")
    n.links = n.links.groupby(level=0).agg(
        {"p_nom_opt": "sum", "p_nom": "sum", **other_cols}
    )

    # set link widths
    link_widths_pipes = h2_pipes / linewidth_factor
    link_widths_imports = h2_imports / linewidth_factor
    if link_widths_pipes.notnull().empty:
        print("No base H2 pipeline capacities to plot.")
        return
    link_widths_pipes = link_widths_pipes.reindex(n.links.index).fillna(0.0)
    link_widths_imports = link_widths_imports.reindex(n.links.index).fillna(0.0)

    # drop non H2 buses
    n.buses.drop(
        n.buses.index[
            (~n.buses.carrier.str.contains("H2")) | (n.buses.carrier.str.contains("OH"))
        ],
        inplace=True,
    )

    # optionally add hydrogen storage capacities onto the map
    if regions_for_storage is not None:
        h2_storage = n.stores.query("carrier.str.contains('H2')")
        regions_for_storage["H2"] = (
            h2_storage.rename(index=h2_storage.bus.map(n.buses.location))
            .e_nom_opt.groupby(level=0)
            .sum()
            .div(1e6)
        )  # TWh
        regions_for_storage["H2"] = regions_for_storage["H2"].where(
            regions_for_storage["H2"] > 0.1
        )

    # plot H2 pipeline capacities and imports
    print("Plotting base H2 pipeline and import capacities.")
    fig, ax = plt.subplots(figsize=figsize, subplot_kw={"projection": proj})

    n.plot.map(
        geomap=True,
        bus_sizes=0.1,
        bus_colors=color_h2_node,
        link_colors=color_h2_pipe,
        link_widths=link_widths_pipes,
        branch_components=["Link"],
        ax=ax,
        **map_opts,
    )

    if regions_for_storage is not None:
        regions_for_storage = regions_for_storage.to_crs(proj.proj4_init)
        regions_for_storage.plot(
            ax=ax,
            column="H2",
            cmap="Blues",
            linewidths=0,
            legend=True,
            vmax=6,
            vmin=0,
            legend_kwds={
                "label": "Hydrogen Storage [TWh]",
                "shrink": 0.7,
                "extend": "max",
            },
        )

    if not h2_imports.empty:
        n.plot.map(
            geomap=True,
            bus_sizes=0,
            link_colors=color_h2_imports,
            link_widths=link_widths_imports,
            branch_components=["Link"],
            ax=ax,
            **map_opts,
        )

    sizes = [30, 10]
    labels = [f"{s} GW" for s in sizes]
    scale = 1e3 / linewidth_factor
    sizes = [s * scale for s in sizes]

    legend_kw = dict(
        loc="upper left",
        bbox_to_anchor=(0.32, 1.13),
        frameon=False,
        ncol=1,
        labelspacing=0.8,
        handletextpad=1,
    )

    add_legend_lines(
        ax,
        sizes,
        labels,
        patch_kw=dict(color="lightgrey"),
        legend_kw=legend_kw,
    )

    legend_kw = dict(
        loc="upper left",
        bbox_to_anchor=(0.55, 1.13),
        labelspacing=0.8,
        handletextpad=0,
        frameon=False,
    )

    add_legend_circles(
        ax,
        sizes=[0.2],
        labels=["H2 Node"],
        srid=n.srid,
        patch_kw=dict(facecolor=color_h2_node),
        legend_kw=legend_kw,
    )

    colors = (
        [color_h2_pipe, color_h2_imports] if not h2_imports.empty else [color_h2_pipe]
    )
    labels = ["H2 Pipeline", "H2 import"] if not h2_imports.empty else ["H2 Pipeline"]

    legend_kw = dict(
        loc="upper left",
        bbox_to_anchor=(0, 1.13),
        ncol=1,
        frameon=False,
    )

    add_legend_patches(ax, colors, labels, legend_kw=legend_kw)

    ax.set_facecolor("white")

    plt.show()

And plot the H2 reference grid together with the import corridors:

In [ ]:
n = n_NT_2030.copy()


def load_projection(plotting_params):
    proj_kwargs = plotting_params.get("projection", dict(name="EqualEarth"))
    proj_func = getattr(ccrs, proj_kwargs.pop("name"))
    return proj_func(**proj_kwargs)


proj = load_projection(dict(name="EqualEarth"))

map_opts = {
    "boundaries": [-11, 30, 34, 71],
    "geomap_colors": {
        "ocean": "white",
        "land": "white",
    },
}

if n.buses.country.isin(["MA", "DZ"]).any():
    map_opts["boundaries"] = list(np.add(map_opts["boundaries"], [0, 0, -6, 0]))

plot_h2_map_base(n, map_opts, proj)

# Benchmarking framework

In [ ]:
# Present metrics used (incl. reference to methodology)
# Data sources used for comparison
# Mention introduction of onwind and solar
# Showcase current status
# --- Table
# --- Graphs

# Wrap up

In [ ]:
# Collect feedback via Slido